# Metadata Explorer

__Description__: Opens the metadata file created by `PM-EventsTable.ipynb` or `distalEventsTable.ipynb` and explores the metadata associated with the excess rainfall calculation. 

__Input__: The metadata *.json file created by `PM-EventsTable.ipynb` or `distalEventsTable.ipynb`.

__Output__: User-specified metadata as a *.csv.

---



### Load Libraries:

In [1]:
import sys
sys.path.append('../../core')
from hydromet import*

### Specify the Filename and Path:

In [2]:
root_dir = pl.Path(r"C:\Users\sputnam\Documents\Projects\PFRA\PFRA_Production\Pluvial\DC_75_all\Outputs")
outputs_dir = pl.Path(r"C:\Users\sputnam\Desktop")

In [3]:
files = []
for file in root_dir.glob('**/*.json'):
    if 'Metadata' in file.stem and 'SW' not in file.stem:
        files.append(file)
files

[WindowsPath('C:/Users/sputnam/Documents/Projects/PFRA/PFRA_Production/Pluvial/DC_75_all/Outputs/DC_P01_Metadata/DC_P01_D01_Metadata.json'),
 WindowsPath('C:/Users/sputnam/Documents/Projects/PFRA/PFRA_Production/Pluvial/DC_75_all/Outputs/DC_P01_Metadata/DC_P01_D02_Metadata.json'),
 WindowsPath('C:/Users/sputnam/Documents/Projects/PFRA/PFRA_Production/Pluvial/DC_75_all/Outputs/DC_P03_Metadata/DC_P03_D01_Metadata.json'),
 WindowsPath('C:/Users/sputnam/Documents/Projects/PFRA/PFRA_Production/Pluvial/DC_75_all/Outputs/DC_P03_Metadata/DC_P03_D02_Metadata.json'),
 WindowsPath('C:/Users/sputnam/Documents/Projects/PFRA/PFRA_Production/Pluvial/DC_75_all/Outputs/DC_P04_Metadata/DC_P04_D01_Metadata.json'),
 WindowsPath('C:/Users/sputnam/Documents/Projects/PFRA/PFRA_Production/Pluvial/DC_75_all/Outputs/DC_P06_Metadata/DC_P06_D01_Metadata.json'),
 WindowsPath('C:/Users/sputnam/Documents/Projects/PFRA/PFRA_Production/Pluvial/DC_75_all/Outputs/DC_P06_Metadata/DC_P06_D02_Metadata.json')]

In [6]:
durations = ['H06', 'H12', 'H24', 'H96']

writer = pd.ExcelWriter(outputs_dir/'FFRD_DC_Events_Summary.xlsx')
for file in files:
    results = {}
    project_area = file.stem.split('_')[0]
    model = file.stem.split('_')[1]
    domain = file.stem.split('_')[2]
    with open(file) as f:
        metadata = json.load(f)
        
    for dur in durations:
        events_metadata = metadata[dur]['BCName'][domain]['events_metadata']
        events_metadata_df = pd.DataFrame.from_dict(events_metadata)
        events_metadata_df = events_metadata_df.set_index('EventID')
        groups = metadata[dur]['BCName'][domain]['groups']
        
        for final_event, original_events in groups.items():
            temp_df = events_metadata_df.loc[original_events].copy()
            event = f'{project_area}_{model}_{domain}_{final_event}'
            n_events = len(original_events)
            ri_min = temp_df['Tr'].min()
            ri_max = temp_df['Tr'].max()
            ri_avg = temp_df['Tr'].mean()
            precip_min = temp_df['Random Precipitation'].min()
            precip_max = temp_df['Random Precipitation'].max()
            precip_avg = temp_df['Random Precipitation'].mean()
            cn = events_metadata_df['AMC II CN'].iloc[0]
            cn_min = temp_df['Random CN'].min()
            cn_max =  temp_df['Random CN'].max()
            cn_avg = temp_df['Random CN'].mean()
            if cn_avg<cn:
                cn_relative = 'Drier'
            elif cn_avg==cn:
                cn_relative = 'Average'
            else: 
                cn_relative = 'Wetter'
            results[final_event] = {'Original Name': event, 'Num Events, [-]': n_events, 'Duration, [hr]': dur, 
                                    'RI Min, [yr]': ri_min, 'RI Max, [yr]': ri_max, 'RI Avg, [yr]': ri_avg, 
                                    'Depth Min, [in]': precip_min, 'Depth Max, [in]': precip_max, 'Depth Avg, [in]': precip_avg, 
                                    'CN Min, [-]': cn_min, 'CN Max, [-]': cn_max, 'CN Avg, [-]': cn_avg, 'Relative Wetness': cn_relative}
    df = pd.DataFrame.from_dict(results).T        
    df.to_excel(writer, sheet_name = f'{project_area}_{model}_{domain}')
writer.save()

---
## End